In [2]:
# Import standard libraries
import pandas as pd
import numpy as np
import urllib.request
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Import from mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Association Rules

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*4yFCbNwp0gGdGR5KbquFHA.png' width="600">

Source: [Comparing Association Rule Mining with other similar methods](https://medium.com/@utkarsh.kant/comparing-association-rule-mining-with-other-similar-methods-d964eaafad91), by Utkarsh Kant


## Content

The goal of this walkthrough is to provide you with insights on association rules. After presenting the main concepts, you will be introduced to the techniques to implement association rule mining in Python. Finally, it will be your turn to practice, using an application on groceries purchase.

This notebook is organized as follows:
- [Background](#Background)
    - [Objective](#Objective)
    - [Concepts](#Concepts)
    - [Apriori algorithm](#Apriori-algorithm)
- [Implementation](#Implementation)
    - [Discover dataset](#Discover-dataset)
    - [Preprocessing](#Preprocessing)
    - [Applying Apriori algorithm](#Applying-Apriori-algorithm)
    - [Mining Association Rules](#Mining-Association-Rules)
- [Your turn!](#Your-turn!)

## Background

### Objective

[Association rule](https://en.wikipedia.org/wiki/Association_rule_learning) aims at discovering interesting relations between variables in large dataset. Like clustering, association rule mining is an **unsupervised learning** method. However, while clustering techniques calculate clusters based on similarities, association rule finds associations based on co-occurrences.

### Concepts

Our goal is to learn a rule $\Rightarrow$ informing us that, when a set of items $S$ *occur together*, another item $i$ *frequently occurs with them*: $S \Rightarrow i$. Note that **$\Rightarrow$ does not indicate a causal link**.

The most important relationships can be identified using the *support* and *confidence*:
- The **support** indicates how frequently the itemset appears in our dataset, i.e., it measures the notion *occur together*:
$$\text{support}_{S \Rightarrow i}=\frac{\text{# observations containing }S\text{ and }i}{\text{total number of observations}}$$
- The **confidence** measures how frequently item $i$ appears with the set of items $S$, i.e., the notion *frequently occurs with them*:
$$\text{confidence}_{S \Rightarrow i}=\frac{\text{# observations containing }S\text{ and }i}{\text{# observations containing }S}$$

We need both the support and confidence to satisfy a minimum *threshold*. Indeed:
- a low support indicates that the relation can happen by chance and may not be generalized.
- a low confidence indicates that the rule is not reliable.

One drawback of the confidence is that $S \Rightarrow i$ can have a high confidence because item $i$ appears frequently, not because it is associated with $S$. To better measure the interestingness of a rule, we can use the **lift**:
$$\text{lift}_{S \Rightarrow i}=\frac{\frac{\text{# observations containing }S\text{ and }i}{\text{# observations containing }S}}{\frac{\text{# observations containing }i}{\text{# total observations}}}$$

### Apriori algorithm

[Apriori](https://en.wikipedia.org/wiki/Apriori_algorithm) is an algorithm for frequent item set mining and association rule learning, proposed by Agrawal and Srikant in 1994.

The main idea of Apriori is that the subsets of a frequent itemset must also be frequent.
$$\text{For all sets } X,Y, \text{ if } (X \subseteq Y) \text{ then support}(X) \geq \text{support}(Y) $$
Reciprocally, if a itemset is not frequent, then its supersets cannot be frequent.

Hence, instead of computing the support of each itemset, which would be computationally expensive, Apriori uses a "bottom up" approach, where frequent subsets are extended one item at a time and tested, while infrequent itemset and all their supersets are pruned, i.e., not considered.

*Reference:* Agrawal, Rakesh, and Ramakrishnan Srikant. "[Fast algorithms for mining association rules](https://www.it.uu.se/edu/course/homepage/infoutv/ht08/vldb94_rj.pdf)" Proc. 20th int. conf. very large data bases, VLDB. Vol. 1215. 1994



## Implementation

We will implement the Apriori algorithm to mine the frequent itemsets. The `mlxtend` library has an implementation of this algorithm [Documentation](http://rasbt.github.io/mlxtend/). You can install the library using `pip` or `conda`:

```python
!pip install mlxtend
```

### Discover dataset

We are going to use a dataset containing the purchase of customers, available in the /data folder of the course repository.

Source: [Harsh-Git-Hub](https://gist.github.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751)

In [2]:
url_retail ='https://raw.githubusercontent.com/michalis0/MGT-502-Data-Science-and-Machine-Learning/main/data/retail.csv'
retail = pd.read_csv(url_retail, sep=',')
retail.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


Each row of the dataset represents items that were purchased together by a customer, on the same day at the same store.

The dataset is **sparse**, as a relatively high percentage of cells is null (NA, NaN or equivalent). These null values make it difficult to read the table. Let's find out which unique items can actually be found in the table (based on the first column):

In [3]:
# Unique items in first column:
items = retail['0'].unique()

# Print result - we use the join method to print items one by one:
print('Our dataset contains the following items: '
      +', '.join(items))

Our dataset contains the following items: Bread, Cheese, Meat, Eggs, Wine, Bagel, Pencil, Diaper, Milk


### Preprocessing

To make use of the `apriori` module given by `mlxtend` library, we need to convert the dataset to the appropriate format. The `apriori` module requires a dataframe that has either 0 and 1 or True and False as data. Since the data we have is all strings (names of items), we need to encode the data.

We first convert our dataframe to a list of list, removing the NaN values:

In [4]:
# Convert dataframe to list of list
retail_list = retail.values.tolist()

# Remove NaNs with list comprehensions
retail_list_cleaned = [[x for x in y if str(x) != 'nan'] for y in retail_list]

Let's check the results for a few transactions:

In [5]:
print(retail_list_cleaned[0])
print(retail_list_cleaned[4])

['Bread', 'Wine', 'Eggs', 'Meat', 'Cheese', 'Pencil', 'Diaper']
['Meat', 'Pencil', 'Wine']


Next, we use the `TransactionEncoder` module of the `mlxtend` library to transform the transactions to `True` or `False` ([Documentation](http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/)). The module is imported at the beginning of the notebook with the following line of code:

```python
from mlxtend.preprocessing import TransactionEncoder
```

In [6]:
# Create instance of Encoder
te = TransactionEncoder()

# Fit encoder and transform our list
retail_list_encoded = te.fit(retail_list_cleaned).transform(retail_list_cleaned)

# Create dataframe with results
retail_encoded = pd.DataFrame(retail_list_encoded, columns=te.columns_)
retail_encoded.head()

,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine
0,False,True,True,True,True,True,False,True,True
1,False,True,True,True,False,True,True,True,True
2,False,False,True,False,True,True,True,False,True
3,False,False,True,False,True,True,True,False,True
4,False,False,False,False,False,True,False,True,True


### Applying Apriori algorithm

We will now implement the Apriori algorithm using the `apriori` module of the `mlxtend` library to find the frequent itemsets ([Documentation](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/)). Here is the import line:

```python
from mlxtend.frequent_patterns import apriori
```

Here are some of the parameters of the module:
- `df` : DataFrame that has 0 and 1 or True and False as values
- `min_support` : Floating point value between 0 and 1 that indicates the minimum support required for an itemset to be selected.
- `use_colnames` : Allows to preserve column names for itemset making it more readable.
- `max_len` : Max length of itemset generated. If not set, all possible lengths are evaluated.

As output, we obtain a DataFrame with columns 'support' and 'itemsets' of all itemsets that have a support greater than `min_support` and a length strictly lower than `max_len`.

Let's try with a minimum support of 0.2 and no maximum length:

In [7]:
# Apriori algorithm
freq_items = apriori(retail_encoded, min_support=0.2, use_colnames=True)
freq_items.head(15)

,support,itemsets
0,0.425397,(Bagel)
1,0.504762,(Bread)
2,0.501587,(Cheese)
3,0.406349,(Diaper)
4,0.438095,(Eggs)
5,0.476190,(Meat)
6,0.501587,(Milk)
7,0.361905,(Pencil)
8,0.438095,(Wine)
9,0.279365,"(Bread, Bagel)"


### Mining Association Rules

We will now mine association rules using the `association_rules` module of the `mlxtend` library  ([Documentation](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/)). Here is the import line:

```python
from mlxtend.frequent_patterns import association_rules
```

As you know by now, frequent if-then associations are called "association rules". They consist of an antecedent (if) and a consequent (then): `{antecedent} => {consequent}`.

The `association_rules` module requires as input parameters a DataFrame of frequent itemsets as well as:
- `metric` : metric to evaluate if a rule is of interest; can be set to "support", "confidence", "lift", "leverage" and "conviction". See the documentation for more information on how these metrics are defined.
- `min_threshold` : minimal threshold for the evaluation metric to decide whether a candidate rule is of interest.

We obtain as output a DataFrame with columns "antecedents" and "consequents" that store itemsets, plus the scoring metric columns: "antecedent support", "consequent support", "support", "confidence", "lift", "leverage", "conviction" of all rules for which the `metric` is greater than the `min_thresold`. 

Let's try using the the confidence metric with a threshold of 0.6, i.e., we are only keeping rules with a confidence at or above 0.6:

In [8]:
# Generate rules
rules = association_rules(freq_items, metric="confidence", min_threshold=0.6)

# Rules sorted by lift
rules.sort_values(by="lift", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
12,"(Milk, Meat)",(Cheese),0.244444,0.501587,0.203175,0.831169,1.657077,0.080564,2.952137,0.524816
9,"(Eggs, Meat)",(Cheese),0.266667,0.501587,0.215873,0.809524,1.613924,0.082116,2.616667,0.518717
10,"(Cheese, Meat)",(Eggs),0.323810,0.438095,0.215873,0.666667,1.521739,0.074014,1.685714,0.507042
8,"(Eggs, Cheese)",(Meat),0.298413,0.476190,0.215873,0.723404,1.519149,0.073772,1.893773,0.487091
11,"(Cheese, Milk)",(Meat),0.304762,0.476190,0.203175,0.666667,1.400000,0.058050,1.571429,0.410959
1,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203,0.469167
2,(Cheese),(Meat),0.501587,0.476190,0.323810,0.645570,1.355696,0.084958,1.477891,0.526414
3,(Meat),(Cheese),0.476190,0.501587,0.323810,0.680000,1.355696,0.084958,1.557540,0.500891
0,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650,0.402687
7,(Eggs),(Meat),0.438095,0.476190,0.266667,0.608696,1.278261,0.058050,1.338624,0.387409


The `rules` dataframe contains all the association rules that we determined as interesting. What do you think? Are they really interesting? What does the lift metric tells us?

Use the interactive function below to further explore the above rules with different threshold for confidence. What do you think about the rules when the threshold is 0.4?

In [9]:
metrics = ['lift', 'support', 'confidence']
thresholds = [0.6, 0.5, 0.4]

@interact
def interactive_association(sort_by = metrics, threshold = thresholds):
    rules_interactive = association_rules(freq_items, metric="confidence", min_threshold= threshold)
    return rules_interactive.sort_values(by=sort_by, ascending=False)

/Users/pcw/opt/anaconda3/lib/python3.9/site-packages/ipykernel/pylab/backend_inline.py:10: DeprecationWarning: `ipykernel.pylab.backend_inline` is deprecated, directly use `matplotlib_inline.backend_inline`
  warnings.warn(


interactive(children=(Dropdown(description='sort_by', options=('lift', 'support', 'confidence'), value='lift')…

## Your turn!

Now it's your turn to practice. We will use a bigger dataset containing the groceries purchase of customers. 

Note that this is not a proper CSV file since there are different number of values in each row. Hence, we have to read the file manually.

In [10]:
url_groceries = 'https://raw.githubusercontent.com/michalis0/MGT-502-Data-Science-and-Machine-Learning/main/data/groceries.csv'

# Open and read url, and decode into a string
groceries_str = urllib.request.urlopen(url_groceries).read().decode("utf-8")

# Create a list where each item is one line, i.e., one transaction
groceries_lis = groceries_str.split('\n')

# Create a list of list where each item is one good
groceries = [[item for item in line.split(',')] for line in groceries_lis]

Here is how our processed data looks like:

In [11]:
groceries[0:4]

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'],
 ['tropical fruit', 'yogurt', 'coffee'],
 ['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese', 'meat spreads']]

- Encode the data in a dataframe of True and False

In [12]:
# YOUR CODE HERE

te = TransactionEncoder()
te_groceries = te.fit(groceries).transform(groceries)
df = pd.DataFrame(te_groceries, columns=te.columns_)
df.head()

,,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


- Find association rules for the Groceries dataset using **confidence** as the `metric` parameter, a support threshold of **0.001** and confidence threshold of **0.05**

In [13]:
# YOUR CODE HERE

# Find the frequent itemsets with  min_support=0.001, max_len=2
freq_items = apriori(df, min_support=0.001, use_colnames=True, max_len=2)

# Find the association rules with metric='confidence' and min_threshold=0.05
rules = association_rules(freq_items, metric="confidence", min_threshold=0.05)

- Extract all the rules you have found containing "bottled beer" as *antecedent*. Which rules do you find interesting? Can you explain them (e.g., potato chips may be frequently bought with bottled bears for "apéro")?

In [14]:
# YOUR CODE HERE

rules.loc[rules["antecedents"].astype(str).str.contains("bottled beer")].sort_values(by="lift", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
417,(bottled beer),(liquor),0.080521,0.011082,0.004677,0.058081,5.241127,0.003784,1.049897,0.880065
445,(bottled beer),(red/blush wine),0.080521,0.019215,0.004880,0.060606,3.154080,0.003333,1.044061,0.742758
377,(bottled beer),(bottled water),0.080521,0.110512,0.015758,0.195707,1.770906,0.006860,1.105925,0.473439
382,(bottled beer),(butter),0.080521,0.055409,0.005795,0.071970,1.298888,0.001334,1.017845,0.250262
421,(bottled beer),(margarine),0.080521,0.058560,0.006100,0.075758,1.293666,0.001385,1.018607,0.246882
427,(bottled beer),(napkins),0.080521,0.052359,0.005185,0.064394,1.229862,0.000969,1.012864,0.203268
460,(bottled beer),(soda),0.080521,0.174359,0.016978,0.210859,1.209332,0.002939,1.046252,0.188256
408,(bottled beer),(fruit/vegetable juice),0.080521,0.072285,0.007015,0.087121,1.205238,0.001195,1.016252,0.185201
402,(bottled beer),(frankfurter),0.080521,0.058967,0.005388,0.066919,1.134857,0.000640,1.008522,0.129238
441,(bottled beer),(pork),0.080521,0.057645,0.005185,0.064394,1.117070,0.000543,1.007213,0.113979


- Feel free to further explore various other thresholds and antecedents...

In [15]:
# YOUR CODE HERE
